# TINKING

## 在CTR点击率预估中，使用GBDT+LR的原理是什么？（能简要说明GBDT和LR在CTR预估中的作用）

  特征工程做为机器学习中最重要的一个阶段，在实际应用中由于样本数量很大，而特征数据又很稀疏，此时单独采用LR模型往往会造成模型的欠拟合，面对如此巨大的数据采用人工特征工程，人工成本和时间成本代价都是巨大的，通过机器来实现特征工程的提取成为了必然选择，GDBT+LR模型中GDBT是通过残差概念形成了多棵树，每棵树类似二叉树，每个叶子节点都有唯一的路径，也就是有唯一的一个特征与之相对于，因而它在该模型中被当作是一个特征工程的实现方式，LR是通过sigmod函数对特征工程的进行分类，从而将特征映射到0或者1上面，达到预测效果

## Wide & Deep的模型结构是怎样的，为什么能通过具备记忆和泛化能力（memorization and generalization）（1、能简要说明Wide&Deep的模型 2、如何具备记忆和泛化能力）


wide部分主要就是一个广义线性模型（如LR），它一般处理一阶或者通过FM处理二阶特征，这种较低纬度的数据，数据直接来源于原始数据，因此代表了记忆
deep是深度学习部分如DNN，RNN通过它是特征值进一步分析，处理，寻找出特征之间的内在关系，具有推理能力，因此代表了推理能力，

## 在CTR预估中，使用FM与DNN结合的方式，有哪些结合的方式，代表模型有哪些？

根据计算方式来份主要有两类：
    串行计算：将FM的一次项与二次项结果作为DNN的输入项，让DNN运行，从而得到最终结果，代表模型有NFM，PNN，AFM
    并行计算：FM和DNN部分分开计算，只在输出层进行一次融合，有deepFM，Wide&Deep，DCN
    



## GBDT和随机森林都是基于树的算法，它们有什么区别？（能简要说明这两种基于树的算法的不同）
 

GBDT和随机森林的相同点：
1、都是由多棵树组成

2、最终的结果都是由多棵树一起决定

GBDT和随机森林的不同点：
1、GDBT是通过残差来实现的，而随机森林是每次随机取数据集中的一部分训练，然后再放该数据集放回去
 
2、组成随机森林的树可以是分类树，也可以是回归树；而GBDT只由回归树组成

3、组成随机森林的树可以并行生成；而GBDT只能是串行生成(因为是通过残差来实现的)

4、对于最终的输出结果而言，随机森林采用多数投票等；而GBDT则是将所有结果累加起来，或者加权累加起来

5、随机森林对异常值不敏感，GBDT对异常值非常敏感

6、随机森林对训练集一视同仁，GBDT是基于权值的弱分类器的集成

7、随机森林是通过减少模型方差提高性能，GBDT是通过减少模型偏差提高性能

 ## item流行度在推荐系统中有怎样的应用 （1、冷启动中的使用 2、协同过滤中的TopN推荐 3、其他使用）
 

基于热门推荐，可以集解决冷启动问题

由于热门商品对个人区分度比较小，所以在协同过滤中可以依照热门程度降低它在个人行为中的权重，使得推荐系统能够推荐个人爱好兴趣

# Action

## 使用Wide&Deep模型对movielens进行评分预测

In [8]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import WDL

# from deepctr.inputs import SparseFeat,get_feature_names
# DeepCTR 采用的是  v0.8.0 变动详情
# https://github.com/shenweichen/DeepCTR
from deepctr.feature_column import SparseFeat,get_feature_names

data = pd.read_csv("./code/wide_deep/movielens_sample.txt")
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用WDL进行训练
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用WDL进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
print(target,test[target].values)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

1/1 [==============================] - 1s 1s/step - loss: 14.7499 - mse: 14.7499 - val_loss: 13.1163 - val_mse: 13.1163
['rating'] [[2]
 [4]
 [2]
 [4]
 [3]
 [3]
 [5]
 [4]
 [5]
 [2]
 [4]
 [3]
 [4]
 [2]
 [3]
 [3]
 [2]
 [2]
 [3]
 [3]
 [5]
 [3]
 [5]
 [3]
 [1]
 [3]
 [4]
 [5]
 [4]
 [4]
 [3]
 [4]
 [5]
 [2]
 [2]
 [4]
 [5]
 [5]
 [4]
 [3]]
test RMSE 3.5810752575169373
